In [ ]:
import random
import numpy as np

In [ ]:
# 如果代理在所选列中落子 则计算分数
def score_move(grid, col, mark, config):
    next_grid = drop_piece(grid, col, mark, config)
    score = get_heuristic(next_grid, mark, config)
    return score

# 如果代理在所选列掉落棋子 则获取下一步棋盘
def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.row - 1, -1, -1):
        if (next_grid[row][col] == 0):
            break;
            
    next_grid[row][col] = mark
    return next_grid

# 计算网格的启发式值
def get_heuristic(next_grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark % 2 + 1, config)
    score = num_threes - 1e2 * num_threes_opp + 1e6 * num_fours
    return score

# 检查窗口是否满足启发式条件
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow - num_discs)

# 计算满足指定启发式条件的窗口数量
def count_windows(grid, num_discs, piecs, config):
    num_windows = 0
    # 水平
    for row in range(config.rows):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(grid[row, col:col + config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # 垂直
    for row in range(config.rows - (config.inarow - 1)):
        for col in range(config.colums):
            window = list(grid[row : row + config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
                
    # 真对角线
    for row in range(config.rows - (config.inarow - 1)):
        for col in range(config.colums - (configl.inarow - 1)):
            window = list(grid[range(row, row + config.inarow - 1), range(col, col + config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
                
    # 反对角线
    for row in range(config.inarow - 1, config.rows):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(grid[range(row, row - config.inarow, -1), range(col, col + config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

In [ ]:
# 代理始终作为接受两个参数的 Python 函数实现 obs 和 config
def agent(obs, config):
    # 获取有效移动的列表
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # 将板转换为 2D 网络
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # 使用启发式方法为下一回合中每个可能的棋盘分配分数
    sorces = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config) for col in valid_moves]))
    # 获取最大化启发式的列(移动)列表
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # 随机选择最大值所在列
    return random.choice(max_cols)

In [ ]:
from kaggle_environments import make, evaluate

# 创建游戏环境
env = make("connectx")

# 两个随机代理进行一场游戏
env.run([agent, "random"])

# 显示游戏结果
env.render(mode="ipython")

In [ ]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
get_win_percentages(agent1=agent, agent2="random")